<a href="https://colab.research.google.com/github/OVP2023/ML/blob/main/simple_NN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
from tqdm import tqdm_notebook

Классификация изображений на датасэте данных CIFAR10 из Keras

In [15]:
import numpy as np
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=10**4, random_state=42)

class_names = np.array(['airplane','automobile ','bird ','cat ','deer ','dog ','frog ','horse ','ship ','truck'])

print (X_train.shape,y_train.shape)

(40000, 32, 32, 3) (40000, 1)


In [3]:
X_train = X_train / 255
X_val = X_val / 255
X_test = X_test / 255



Для начала реализуем простую нейросеть:

    принимает на вход картинки размера 32 x 32 x 3;
    вытягивает их в вектор (keras.layers.Flatten);
    пропускает через 1 или 2 полносвязных слоя;
    выходной слой отдает вероятности принадлежности к каждому из 10 классов.


In [5]:
model = tf.keras.models.Sequential([
        tf.keras.Input(shape=X_train.shape[1:]),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')

])

In [6]:
#callbacks  используются для получения метрик обучения и влияния на ход обучения
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.keras',monitor='val_accuracy',save_best_only=True),   # Сохраняется ПРЕДпоследняя лучшая модель с мониторингом по val_accuracy
             tf.keras.callbacks.CSVLogger('training.log'),   #записывает данные обучения по эпохам в CSV файл
             tf.keras.callbacks.EarlyStopping(patience=4)]  #прекращает обучение если patience=7 эпох без улучшения (val_accuracy)

In [10]:
model.compile(
    optimizer = tf.keras.optimizers.RMSprop(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['sparse_categorical_accuracy']
    )
model.fit(X_train, y_train, batch_size=16, epochs=4, callbacks=callbacks, validation_split=0.2)

Epoch 1/4
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 35s 17ms/step - loss: 1.9616 - sparse_categorical_accuracy: 0.2902 - val_loss: 1.6621 - val_sparse_categorical_accuracy: 0.4046
Epoch 2/4


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - loss: 1.5990 - sparse_categorical_accuracy: 0.4289 - val_loss: 1.5776 - val_sparse_categorical_accuracy: 0.4437
Epoch 3/4
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - loss: 1.5251 - sparse_categorical_accuracy: 0.4546 - val_loss: 1.5456 - val_sparse_categorical_accuracy: 0.4465
Epoch 4/4
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - loss: 1.4623 - sparse_categorical_accuracy: 0.4836 - val_loss: 1.5285 - val_sparse_categorical_accuracy: 0.4563


In [13]:
from sklearn.metrics import accuracy_score

predict_x=model.predict(X_test)
classes_x=np.argmax(predict_x,axis=1)

test_acc = accuracy_score(y_test, classes_x)
print("\n Test_acc =", test_acc)
assert test_acc > 0.45, "Not good enough. Back to the drawing board :)"
print(" Not bad!")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step

 Test_acc = 0.4596
 Not bad!
